In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('word_count').config('spark.some.config.option',"some-value").\
getOrCreate()

23/03/10 21:09:58 WARN Utils: Your hostname, avijit-HP-Laptop-15q-bu0xx resolves to a loopback address: 127.0.1.1; using 192.168.18.7 instead (on interface wlo1)
23/03/10 21:09:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/10 21:10:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df1 = spark.range(2,10000000,2)
df2 = spark.range(2,10000000,4)

step1 = df1.repartition(5)
step12 = df2.repartition(6)

step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12,['id'])
step4 = step3.selectExpr("sum(id)")
step4.collect()

[Row(sum(id)=2500000000000)]

In [5]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#22L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=264]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#22L)])
            +- *(6) Project [id#22L]
               +- *(6) SortMergeJoin [id#22L], [id#16L], Inner
                  :- *(4) Sort [id#22L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#22L, 200), ENSURE_REQUIREMENTS, [plan_id=135]
                  :           +- *(3) Project [(id#14L * 5) AS id#22L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=68]
                  :                    +- *(1) Range (2, 10000000, step=2, splits=1)
             